# Fine-tuning a Sequence Classification Model Exam

In this exam, you will be tasked with performing dataset preprocessing and fine-tuning a model for sequence classification. Complete each step carefully according to the instructions provided.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `aubmindlab/bert-base-arabertv02` for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/sanad_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import pipeline

In [3]:
df = pd.read_csv("hf://datasets/CUTD/sanad_df/sanad_df.csv")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [24]:
df.head(10)

,text,label
0,الشارقة - محمد ولد محمد سالمعرضت مساء أمس الأو...,Culture
1,عبدالحكيم الزبيدي شاعر وقاص وناقد، جاءت نصوصه ...,Culture
2,انطلقت في مثل هذه الأيام من العام الفائت فعالي...,Culture
3,أقيمت مساء أمس الأول في إكسبو الشارقة ندوة حوا...,Culture
4,باسمة يونس حينما قال صاحب السموّ الشيخ الدكتور...,Culture
5,قرر اتحاد الأدباء والكتّاب الموريتانيين عقد مؤ...,Culture
6,تتناول الرواية الإماراتية المرأة ومكانتها في ا...,Culture
7,أعلنت هيئة دبي للثقافة والفنون (دبي للثقافة)، ...,Culture
8,توفي أمس الأديب الإسباني ميجيل ديليبس الذي يعت...,Culture
9,نظم مركز الشارقة للشعر الشعبي في مجلس ضاحية مغ...,Culture


In [22]:
print(df['text'][2])

انطلقت في مثل هذه الأيام من العام الفائت فعاليات مهرجان دبي لمسرح الشباب، تحت مظلة مجلس دبي الثقافي، وكانت فكرة المهرجان تهدف الى تشجيع ودعم المواهب الشابة من الفنانين المسرحيين، وإتاحة الفرصة أمامهم لتقديم أعمالهم المسرحية أمام الجمهور، وهو ما يزيد من خبرتهم ويصقل موهبتهم المسرحية، ويوم 10 أغسطس الفائت انطلقت فعاليات المهرجان في دورته الثانية وتستمر حتى 20 أغسطس/ آب الجاري، ولكن مع قرار صاحب السمو الشيخ محمد بن راشد آل مكتوم نائب رئيس الدولة رئيس مجلس الوزراء حاكم دبي، بإنشاء هيئة دبي للثقافة والفنون تحولت رعاية المهرجان الى الهيئة التي تسعى الى المساهمة في الحياة الاجتماعية والفكرية والثقافية في دولة الإمارات، وترسيخ مكانة دبي كمقر حيوي للثقافة والفنون والتراث.وقد وضعت هيئة دبي للثقافة والفنون في استراتيجيتها الثقافة الشاملة، أهمية المحافظة على الهوية الوطنية من خلال تعزيز الوعي بالأنشطة الثقافية، والتعاون مع المؤسسات المعنية في الدولة، للمشاركة في أنشطة وفعاليات الهيئة، والتي يبرز مهرجان دبي لمسرح الشباب في مقدمتها، لأن الهيئة تؤمن بضرورة أن يظل المسرح يقوم بدوره العضوي الحيوي في ال

In [4]:
df.shape

(15000, 3)

In [3]:
df['label'].unique()

array(['Culture', 'Finance', 'Medical'], dtype=object)

## Step 2: Clean Unnecessary Columns

Remove any columns from the dataset that are not needed for training.

In [5]:
df.drop(columns=['Unnamed: 0'], inplace= True)

## Step 3: Splitting the Dataset

Split the dataset into training and testing sets, ensuring that 20% of the data is used for testing.

In [6]:
train_df , test_df = train_test_split(df, test_size= 0.2, random_state=42)

In [7]:
le = LabelEncoder()

train_df['label'] = le.fit_transform(train_df['label'])
test_df['label'] = le.fit_transform(test_df['label'])

In [6]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [8]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

## Step 4: Tokenizing the Data

Initialize a tokenizer for the model.

In [9]:
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")

def tokenize_fun(examples):
  return tokenizer(examples['text'], truncation = True, padding = True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Step 5: Preprocessing the Text

Map the tokenization function to the dataset. Ensure the text data is processed using truncation to handle sequences that exceed the model's input size. Please do any further preprocessing.

**Bonus**: If you performed more comprehensive preprocessing, such as removing links, converting text to lowercase, or applying additional preprocessing techniques.

In [10]:
tokenized_train_dataset = train_dataset.map(tokenize_fun, batched = True)
tokenized_test_dataset = test_dataset.map(tokenize_fun, batched= True)

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

### Step 6: Label Encoding

Convert the categorical labels into numerical format using a label encoder if needed.

### Step 7: Data Collation for Padding

Prepare the data for training by ensuring all sequences in a batch are padded to the same length. Use a data collator to handle dynamic padding.

In [11]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

### Step 8: Model Initialization

Initialize a sequence classification model using the BERT-based architecture. Set the the right amount of output labels.

In [12]:
checkpoint = 'aubmindlab/bert-base-arabertv02'
num_labels = len(df['label'].unique())

config = AutoConfig.from_pretrained(checkpoint, num_labels= num_labels)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config = config)

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step 9: Training Arguments

Define the training arguments, including parameters such as learning rate, batch size, number of epochs, and weight decay.

In [14]:
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate = 4e-5,
    per_device_train_batch_size=4,
    num_train_epochs=1,
    evaluation_strategy="no"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Step 10: Trainer Initialization and Training

Set up the trainer with the model, training arguments, dataset, tokenizer, and data collator. Train the model using the dataset you processed earlier.

In [20]:
tranier = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train_dataset
    data_collator= data_collator
)
tranier.train()

Step,Training Loss
500,0.057400
1000,0.031300
1500,0.024300
2000,0.027100
2500,0.023300
3000,0.020600


TrainOutput(global_step=3000, training_loss=0.03065721416473389, metrics={'train_runtime': 1447.1861, 'train_samples_per_second': 8.292, 'train_steps_per_second': 2.073, 'total_flos': 3157361012736000.0, 'train_loss': 0.03065721416473389, 'epoch': 1.0})

In [21]:
model.save_pretrained("./mymodel")
tokenizer.save_pretrained("./mytokenizer")

('./mytokenizer/tokenizer_config.json',
 './mytokenizer/special_tokens_map.json',
 './mytokenizer/vocab.txt',
 './mytokenizer/added_tokens.json',
 './mytokenizer/tokenizer.json')

## Step 11: Inference

Once the model is trained, perform inference on a sample text to evaluate the model's prediction capabilities. Use the tokenizer to process the text, and then feed it into the model to get the predicted label.

**Examples on Medical article:**

In [ ]:
model_inf = pipeline("sentiment-analysis", model = "/content/mymodel", tokenizer="/content/mytokenizer",return_all_scores=False)

In [25]:
example1 ="""

رأس الخيمة: «الخليج» افتتح الدكتور عبدالله النعيمي، مدير منطقة رأس الخيمة الطبية، مركز الظيت للرعاية الصحية الأولية، الذي استأنف عمله بعد إنجاز مشروع التطوير والصيانة، الذي خضع له، وهو يخدم منطقة حيوية تضم كثافة سكانية عالية. وأوضح د. النعيمي، خلال زيارته للمركز الصحي، أن أعمال الصيانة للمركز بدأت في يناير/كانون الثاني من العام الماضي، واشتملت على تعديلات في المبنى وتطوير مرافقه وخدماته، من أهمها إنشاء حجرات إضافية، وأقسام جديدة للمختبر والتمريض والعيادات، وتجهيز المواقف الخارجية لسيارات المراجعين والموظفين، وتعديلات داخلية في المبنى الرئيسي للمركز الصحي الحيوي. وبين النعيمي أن الهدف من مشروع توسعة مركز الظيت وتطويره هو الارتقاء بالخدمات الصحية المقدمة لأهالي المنطقة، وتسهيل تنقل المراجعين بين مرافق المركز، وتسهيل عملية دخول المراجعين بسياراتهم إلى حرم المركز. وأضاف النعيمي أن «الأشغال العامة» تسلمت المبنى من المنطقة الطبية في وقت سابق، وتكفلت بتنفيذ التوسعات والتعديلات المنشودة، ليكون المركز جاهزاً للعمل في إبريل/نيسان الحالي. وأشار الدكتور النعيمي إلى أن المنطقة تقدم خدمات الرعاية الصحية الأولية إلى مراجعيها عبر 18 مركزاً صحياً تنتشر جغرافياً، في مختلف أنحاء الإمارة.



"""

result = model_inf(example1)
print(result)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[{'label': 'LABEL_2', 'score': 0.9999425411224365}]


**Examples on Culture article:**

In [26]:
example2 ="""

نطلقت في مثل هذه الأيام من العام الفائت فعاليات مهرجان دبي لمسرح الشباب، تحت مظلة مجلس دبي الثقافي، وكانت فكرة المهرجان تهدف الى تشجيع ودعم المواهب الشابة من الفنانين المسرحيين، وإتاحة الفرصة أمامهم لتقديم أعمالهم المسرحية أمام الجمهور، وهو ما يزيد من خبرتهم ويصقل موهبتهم المسرحية، ويوم 10 أغسطس الفائت انطلقت فعاليات المهرجان في دورته الثانية وتستمر حتى 20 أغسطس/ آب الجاري، ولكن مع قرار صاحب السمو الشيخ محمد بن راشد آل مكتوم نائب رئيس الدولة رئيس مجلس الوزراء حاكم دبي، بإنشاء هيئة دبي للثقافة والفنون تحولت رعاية المهرجان الى الهيئة التي تسعى الى المساهمة في الحياة الاجتماعية والفكرية والثقافية في دولة الإمارات، وترسيخ مكانة دبي كمقر حيوي للثقافة والفنون والتراث.وقد وضعت هيئة دبي للثقافة والفنون في استراتيجيتها الثقافة الشاملة، أهمية المحافظة على الهوية الوطنية من خلال تعزيز الوعي بالأنشطة الثقافية، والتعاون مع المؤسسات المعنية في الدولة، للمشاركة في أنشطة وفعاليات الهيئة، والتي يبرز مهرجان دبي لمسرح الشباب في مقدمتها، لأن الهيئة تؤمن بضرورة أن يظل المسرح يقوم بدوره العضوي الحيوي في الحراك الاجتماعي والثقافي، وهو ما يسهم في وعي الإنسان وإدراكه لذاته وواقعه ومجتمعه ووطنه.وتسعى الهيئة أيضاً الى دعم وتشجيع الجيل المسرحي الشاب في الإمارات، من خلال مشاركته في فعالياتها، والمساهمة في ترويج منتوجه الثقافي والفني، خصوصاً أن المكانة التي وصل إليها المسرح في الإمارات، على المستويين الخليجي والعربي، تتطلب النظر بمسؤولية الى أهمية وجود جيل مسرحي شاب وموهوب فنياً وفكرياً، يستكمل المسيرة، ويحافظ على المستوى الذي حققه الجيل المفصل، والذي يتربع الآن على القمة.والمتابع لفعاليات مهرجان دبي لمسرح الشباب، بدأ يتعرف من خلال العروض على أسماء شابة في مجال التمثيل والتأليف والإخراج، وأصبحت أسماء مألوفة له، مثل حمد الحمادي وعبدالله الظاهري وخالد علي ربيع ومحمد الحمادي ومنال بن عمرو ومروان عبدالله صالح وهم جميعاً من العاملين في مجال الإخراج وقد تتلمذوا على أيدي كوكبة من المخرجين الإماراتيين من الجيل المفصل وقد برزوا من خلال أعمالهم المسرحية التي تدور حول الأرض والوطن والهوية بدلالات ورموز مختلفة في أبعادها الفنية والفكرية، والموقف من مواكبة العصر أو التمسك بالماضي، وأحلام الإنسان العادي وطموحاته في هذه الحياة التي تعج بالصراعات الاجتماعية والسياسية والاقتصادية.ولا تقتصر فعاليات المهرجان على العروض وحسب، وإنما أدرجت اللجنة الاستشارية للمهرجان وفي كل دورة، تكريم أحد الفنانين المسرحيين من الجيل المؤسس في الدولة، حيث يكرم في هذه الدورة الفنان الشامل عبدالله صالح، وهي لفتة نبيلة تحسب لإدارة المهرجان.نواف يونس

"""

result = model_inf(example2)
print(result)

[{'label': 'LABEL_0', 'score': 0.9999538660049438}]


**Examples on Financial article:**

In [27]:
example3 = '''

وظبي - "الخليج":انخفض مؤشر سوق الإمارات المالي الصادر عن هيئة الأوراق المالية والسلع خلال جلسة تداول أمس، بنسبة 72 .0% ليغلق على 22 .5422 نقطة . وشهدت القيمة السوقية انخفاضاً بقيمة 92 .5 مليار درهم لتصل إلى 99 .813 مليار درهم، وتم تداول ما يقارب 17 .1 مليار سهم بقيمة إجمالية بلغت 25 .3 مليار درهم من خلال 17182 صفقة .وبلغ عدد الشركات التي تم تداول أسهمها 65 من أصل 120 شركة مدرجة في الأسواق المالية . وحققت أسعار أسهم 15 شركة ارتفاعاً في حين انخفضت أسعار أسهم 42 شركة، بينما لم يحدث أي تغير على أسعار أسهم باقي الشركات .وجاء سهم "شركة أرابتك القابضة" في المركز الأول من حيث الشركات الأكثر نشاطاً، حيث تم تداول ما قيمته 82 .0 مليار درهم موزعة على 71 .84 مليون سهم من خلال 2995 صفقة . وجاء سهم "العربية للطيران" في المركز الثاني من حيث الشركات الأكثر نشاطاً، حيث تم تداول ما قيمته 32 .449 مليون درهم موزعة على 288 مليون سهم من خلال 2039 صفقة .وحقق سهم "تكافل الإمارات" أكثر نسبة ارتفاع سعري حيث أقفل سعر السهم على مستوى 17 .1 درهم مرتفعا بنسبة 35 .9% من خلال تداول 05 .12 مليون سهم بقيمة 21 .14 مليون درهم، وجاء في المركز الثاني من حيث الارتفاع السعري سهم "العربية للطيران" ليغلق على مستوى 58 .1 درهم مرتفعاً بنسبة 48 .7% من خلال تداول 288 مليون سهم بقيمة 32 .449 مليون درهم .وسجل سهم "شركة أبوظبي الوطنية للتأمين" أكثر انخفاض سعري في جلسة التداول حيث أقفل سعر السهم على مستوى 30 .6 درهم مسجلاً خسارة بنسبة 10% من خلال تداول 30 ألف سهم بقيمة 189 ألف درهم . تلاه سهم "الشركة الوطنية للتأمينات العامة" الذي انخفض بنسبة 37 .8% ليغلق على مستوى 94 .3 درهم من خلال تداول 300 سهم بقيمة 1182 درهماً .ومنذ بداية العام بلغت نسبة الارتفاع في مؤشر سوق الإمارات المالي 70 .25%، وبلغ إجمالي قيمة التداول 48 .245 مليار درهم . وبلغ عدد الشركات التي حققت ارتفاعاً سعرياً 70 من أصل ،120 وعدد الشركات المتراجعة 35 شركة .

'''

result = model_inf(example3)
print(result)

[{'label': 'LABEL_1', 'score': 0.9999779462814331}]
